<a href="https://colab.research.google.com/github/Apple03244/Colaboratory/blob/main/Tensor_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import google.colab as gc
f=gc.files.upload()

Saving MyKaggle.py to MyKaggle.py


In [2]:
from MyKaggle import startkaggle
startkaggle()

Let's start Kaggle!


In [3]:
!kaggle competitions download -c playground-series-s4e1

100% 6.81M/6.81M [00:00<00:00, 69.7MB/s]
100% 6.81M/6.81M [00:00<00:00, 69.2MB/s]


In [4]:
!unzip /content/playground-series-s4e1.zip
!rm /content/playground-series-s4e1.zip

Archive:  /content/playground-series-s4e1.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [5]:
import pandas as pd
train=pd.read_csv("/content/train.csv")
test=pd.read_csv("/content/test.csv")
submission=pd.read_csv("/content/sample_submission.csv")

## HyperBand

상위 1/Factor 모델만 추리는 방법

위의 과정을 반복하여 단독 모델을 찾아낸다.

In [6]:
import numpy as np
use_columns=train.select_dtypes(np.number).columns.difference(["id","CustomerId"])

In [7]:
train[use_columns].head(3)

,Age,Balance,CreditScore,EstimatedSalary,Exited,HasCrCard,IsActiveMember,NumOfProducts,Tenure
0,33.0,0.0,668,181449.97,0,1.0,0.0,2,3
1,33.0,0.0,627,49503.50,0,1.0,1.0,2,1
2,40.0,0.0,678,184866.69,0,1.0,0.0,2,10


In [8]:
import tensorflow.keras as tk
!pip install -q -U keras-tuner
import keras_tuner as kt
import functools as ft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.2 MB/s eta 0:00:00


In [9]:
class MyClassifier(kt.HyperModel):
  def __init__(self, name=None, tunable=True):
    super().__init__(name, tunable)
  def build(self,hp:kt.HyperParameters):

    input=tk.layers.Input(shape=[len(use_columns)-1,])

    n_hidden=hp.Int("n_hidden",min_value=1,max_value=5)
    n_units=hp.Int("n_units",min_value=16,max_value=512)
    learning_rate=hp.Float("learing_rate",min_value=1e-10,max_value=1e-2,sampling="log")

    Layer=ft.partial(tk.layers.Dense,
                     units=n_units,
                     activation="relu",
                     kernel_initializer="he_normal")

    output=input
    for _ in range(n_hidden):
      output=Layer()(output)

    output=tk.layers.Dense(1,activation="sigmoid")(output)

    Model=tk.Model(inputs=input,outputs=output)

    loss=tk.losses.BinaryCrossentropy()
    optimizer=tk.optimizers.Adam(learning_rate=learning_rate)
    metrics=tk.metrics.BinaryAccuracy()

    Model.compile(loss=loss,optimizer=optimizer,metrics=metrics)
    return Model

  def fit(self,hp,Model,X,y,**kwargs):
    if hp.Boolean("normalize"):
      X=tk.layers.BatchNormalization()(X)
    return Model.fit(X,y,**kwargs)

In [10]:
hyper_band_tuner=kt.Hyperband(MyClassifier(),objective="val_binary_accuracy",seed=10,max_epochs=30,
                              factor=3,hyperband_iterations=2,overwrite=True,directory="BankChurn",project_name="hyper_band")

In [11]:
import pathlib as pth
import time

In [12]:
def get_run_dir(root_logdir="my_log"):
  return pth.Path(root_logdir)/time.strftime("run_%Y_%m_%d_%H_%M_%S")

In [13]:
root_logdir=pth.Path(hyper_band_tuner.project_dir)/'tensorboard'

In [ ]:
tensor=tk.callbacks.TensorBoard(log_dir=root_logdir)
early=tk.callbacks.EarlyStopping(monitor="binary_accuarcy",patience=5)
hyper_band_tuner.search(X=train[use_columns.difference(["Exited"])],y=train['Exited'],epochs=10
                        ,validation_split=0.1,callbacks=[tensor,early])

Trial 59 Complete [00h 02m 24s]
val_binary_accuracy: 0.7898691296577454

Best val_binary_accuracy So Far: 0.7898691296577454
Total elapsed time: 01h 48m 43s

Search: Running Trial #60

Value             |Best Value So Far |Hyperparameter
2                 |2                 |n_hidden
415               |69                |n_units
3.5618e-07        |0.0052715         |learing_rate
False             |False             |normalize
4                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/4
   1/4642 [..............................] - ETA: 1:59:22 - loss: 30054.6875 - binary_accuracy: 0.7812

3467/4642 [=====================>........] - ETA: 7s - loss: 13019.4277 - binary_accuracy: 0.7470